# Catalog Objects
There are two objects for catalogs, `ClCatalogs` for clusters and `MemCatalogs` for cluster members. Both of them have the same properties of `astropy` tables with additional functionality.

In [ ]:
%load_ext autoreload
%autoreload 2

## Common properties of `ClCatalog` and `MemCatalog`
Both catalog objects have the following attributes:
- `name`: ClCatalog name
- `data`: Table with main catalog data (ex: id, ra, dec, z) and matching data (mt_self, mt_other, mt_cross, mt_multi_self, mt_multi_other)
- `mt_input`: Table containing the necessary inputs for the match (added by Match objects)
- `size`: Number of objects in the catalog
- `id_dict`: Dictionary of indicies given the object id
- `labels`: Labels of data columns for plots

### Creating a catalog
To create a catalog, you have to pass the name as the initial argument and the data for the table as keyword arguments:

In [ ]:
from clevar import ClCatalog, MemCatalog
cat = ClCatalog('cluster', id=['c1', 'c2'], mass=[1e13, 1e14])
mem = MemCatalog('members', id=['m1', 'm2', 'm3'], id_cluster=['c1', 'c2', 'c1'])
# Format for nice display
cat['mass'].info.format = '.2e'

`ClCatalog` will always have the matching columns added, while `MemCatalog` will add them when necessary:

In [ ]:
display(cat)
display(mem)

#### Create a catalog from `fits` files
The catalogs objects can also be read directly from file, by passing the fits file as the first argument, the catalog name as the second, and the names of the columns in the fits files as keyword arguments:

In [ ]:
cat = ClCatalog.read('../demo/cat1.fits', 'cluster',
                     id='ID', mass='MASS')
mem = MemCatalog.read('../demo/cat1_mem.fits', 'member',
                     id='ID', id_cluster='ID_CLUSTER')

### Catalog data
All catalogs have an `id` column if not included in the input, one will be created:

In [ ]:
cat = ClCatalog('cluster', mass=[1e13, 1e14])
cat

Almost all keyword arguments will become columns of the catalog (see exeptions in [Properties of `ClCatalog`](#clcat) and [Properties of `MemCatalog`](#memcat)):

In [ ]:
cat = ClCatalog('test name', test_column=[1, 2], other=[True, False], third=[None, []])
cat

The catalogs have a `label` attibute that is used for plots. If it is not provided as argument, a default value is assigned:

In [ ]:
cat = ClCatalog('cluster', id=['c1', 'c2'], mass=[1e13, 1e14])
cat.labels

In [ ]:
cat = ClCatalog('cluster', id=['c1', 'c2'], mass=[1e13, 1e14],
                labels={'id':'cluster ID', 'mass':'cluster M_200'})
cat.labels

To de main data table of the catalog can be accessed with `[]` operations in the same way as `astropy` tables. The output is a new `Catalog` object, exept when only 1 row or column is required:

In [ ]:
cat = ClCatalog('cluster', id=['c1', 'c2'], mass=[1e13, 1e14])

In [ ]:
print(type(cat['id']))
display(cat['id'])

In [ ]:
print(type(cat['id']))
display(cat['id'])

In [ ]:
print(type(cat['id', 'mass']))
display(cat['id', 'mass'])

In [ ]:
print(type(cat[[1, 0]]))
display(cat[[1, 0]])

In [ ]:
print(type(cat[:1]))
display(cat[:1])

In [ ]:
print(type(cat[0]))
display(cat[0])

### Inbuilt function of catalogs
The catalog objects has some inbuilt functionality to facilitate the matching. `ids2inds` returns the indicies of objects given an id list. Other functions are related to footprint computations, see <a href='footprint.ipynb'>footprint.ipynb</a> for information on those.

In [ ]:
cat = ClCatalog('cluster', id=['c1', 'c2'], mass=[1e13, 1e14])
inds = cat.ids2inds(['c2', 'c1'])
display(cat)
display(cat[inds])

## Properties of `ClCatalog`<a id='clcat'/>

As shown above, `ClCatalog` can have any column in its main data table, however there are a few key columns these catalogs must have to be used for matching:

- `id` - necessary in membership matching and must correspond to `id_cluster` in the cluster member catalog.
- `ra` (in degrees) - necessary for proxity matching.
- `dec` (in degrees) - necessary for proxity matching.
- `z` - necessary for proxity matching if used as matching criteria (or for angular to physical convertion).
- `mass` (or mass proxy) - necessary for proxity matching if `shared_member_fraction` used as preference criteria for unique matches (default use in membership matching).
- `radius` - necessary for proxity matching if used as a criteria of matching (also requires `radius_unit` to be passed)

`radius_unit` can be in angular units (`radians`, `degrees`, `arcmin`, `arcsec`) or physical units (`Mpc`, `kpc`, `pc`) or can enven be given by mass overdensity units (`m200b`, `m500c`) and are case insensitive. In the proximity matching the radius is converted to angular distances (degrees):

In [ ]:
from clevar.match import ProximityMatch
from clevar.cosmology import AstroPyCosmology
mt = ProximityMatch()
cosmo = AstroPyCosmology()

In [ ]:
cat = ClCatalog('Cat', radius=[0.01, 0.02], radius_unit='radians')
mt.prep_cat_for_match(cat, delta_z=None, match_radius='cat')
cat.mt_input['ang']

In [ ]:
cat = ClCatalog('Cat', radius=[0.01, 0.02], radius_unit='degrees')
mt.prep_cat_for_match(cat, delta_z=None, match_radius='cat')
cat.mt_input['ang']

In [ ]:
cat = ClCatalog('Cat', radius=[1, 1.5], z=[.4, .5], radius_unit='mpc')
mt.prep_cat_for_match(cat, delta_z=None, match_radius='cat', cosmo=cosmo)
cat.mt_input['ang']

In [ ]:
cat = ClCatalog('Cat', radius=[1e13, 1e14], z=[.4, .5], radius_unit='m200c')
mt.prep_cat_for_match(cat, delta_z=None, match_radius='cat', cosmo=cosmo)
cat.mt_input['ang']

## Properties of `MemCatalog`<a id='memcat'/>

As shown above, `MemCatalog` can have any column in its main data table, however there are a few key columns these catalogs must have to be used for matching:

- `id` - necessary in membership matching of members.
- `id_cluster` - always necessary and must correspond to `id` in the main cluster catalog.
- `ra` (in degrees) - necessary for proxity matching of members.
- `dec` (in degrees) - necessary for proxity matching of members.
- `pmem` - Probability of the galaxy being a member, must be [0, 1]. If not provided, it will assing 1 for all members.

The `MemCatalog` object also has a `id_dict_list` atribute when there is repeated member `id`.